WPP2024_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT.xlsx  
hdro_indicators_aggregates_sdn.csv  
hdro_indicators_sdn.csv  
indicators_sdn.csv  
sdg_indicatorlist_sdn.csv  
sdg_data_sdn.csv  
gender_sdn.csv  

In [3]:
import pandas as pd
import numpy as np
import re

In [4]:
from google.colab import drive
drive.mount('/content/MyDrive')

Mounted at /content/MyDrive


In [5]:
!ls ./MyDrive/MyDrive/omneda_sudan/

dtm_sdn_smu-bi-weekly-13-_-17122024_v02_public_hdx.xlsx
education_sdn.csv
gender_sdn.csv
hdro_indicators_aggregates_sdn.csv
hdro_indicators_sdn.csv
indicators_sdn.csv
primary-secondary-enrollment-completion-rates.csv
sdg_data_sdn.csv
sdg_indicatorlist_sdn.csv
WPP2024_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT.xlsx
WPP2024_GEN_F01_DEMOGRAPHIC_INDICATORS.xlsx


### Read in the United Nations demographic indicator data

###### WPP2024_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT.xlsx

In [6]:
demo_indicator = pd.read_excel('./MyDrive/MyDrive/omneda_sudan/WPP2024_GEN_F01_DEMOGRAPHIC_INDICATORS_COMPACT.xlsx',
                               sheet_name="Estimates",
                               skiprows=15, header=[0,1])

In [7]:
demo_indicator.head()

Unnamed: 0_level_0 Unnamed: 1_level_0                   Unnamed: 2_level_0  \
               Index            Variant Region, subregion, country or area *   
0                  1          Estimates                                World   
1                  2          Estimates                                World   
2                  3          Estimates                                World   
3                  4          Estimates                                World   
4                  5          Estimates                                World   

  Unnamed: 3_level_0 Unnamed: 4_level_0 Unnamed: 5_level_0 Unnamed: 6_level_0  \
               Notes      Location code    ISO3 Alpha-code    ISO2 Alpha-code   
0                NaN                900                NaN                NaN   
1                NaN                900                NaN                NaN   
2                NaN                900                NaN                NaN   
3                NaN                900                NaN                NaN   
4                NaN                900                NaN                NaN   

  Unnamed: 7_level_0 Unnamed: 8_level_0 Unnamed: 9_level_0  ...  \
         SDMX code**               Type        Parent code  ...   
0                1.0              World                  0  ...   
1                1.0              World                  0  ...   
2                1.0              World                  0  ...   
3                1.0              World                  0  ...   
4                1.0              World                  0  ...   

                                                                      Mortality  \
  Male Mortality before Age 60 (deaths under age 60 per 1,000 male live births)   
0                                              580.5                              
1                                            566.566                              
2                                            546.444                              
3                                            535.811                              
4                                            522.058                              

                                                                                     \
  Female Mortality before Age 60 (deaths under age 60 per 1,000 female live births)   
0                                            497.388                                  
1                                            488.435                                  
2                                             475.37                                  
3                                            467.361                                  
4                                            455.621                                  

                                                                                               \
  Mortality between Age 15 and 50, both sexes (deaths under age 50 per 1,000 alive at age 15)   
0                                            238.516                                            
1                                            229.703                                            
2                                            217.311                                            
3                                            211.257                                            
4                                            203.337                                            

                                                                                              \
  Male Mortality between Age 15 and 50 (deaths under age 50 per 1,000 males alive at age 15)   
0                                            268.734                                           
1                                            256.236                                           
2                                             238.56                                           
3                                            230.96

In [8]:
# demo_indicator.columns.values
demo_indicator.columns=['_'.join(col).strip() for col in demo_indicator.columns.values]
demo_indicator.columns=[re.findall("level_0_(.*)", col)[0]  if col.startswith('Unnamed') else col for col in demo_indicator.columns ]

# demo_indicator.columns

In [55]:
demographics=demo_indicator[demo_indicator['Region, subregion, country or area *'].eq('Sudan')]


In [56]:
def mergeIntoDemographics(df, colName='Indicator Name', indexName='Year', valueName='Value', suffix=None ):

  return demographics.merge(df.pivot(columns=colName, values = valueName, index=indexName).reset_index(),
                                 left_on  = 'Year',
                                 right_on = indexName,
                                 how      = 'left',
                                 suffixes=(None, suffix)
                                 )

def cleanFile(df, yearCol='Year', indicatorCol='Indicator Name', valCol="value"):
  df.drop(0, inplace=True)
  df[yearCol]      = df[yearCol].apply(pd.to_numeric)
  df[valCol]       = df[valCol].apply(pd.to_numeric)
  df[indicatorCol] = df[indicatorCol].astype(str)
  df.drop_duplicates(subset=[yearCol, indicatorCol], inplace=True)
  return df


def printStats(filename, df=demographics):
  print(f"filename {filename}, demographics column count {len(df.columns)}, demographics shape {(df.shape)}")
  # print(f"demographics column count {len(df.columns)}")
  # print(f"demographics shape {(df.shape)}")


def cleanAndMerge(path, filename, yearCol, indicatorCol, valCol, suffix ):

  path = path[:-1] if path[-1] == "/" else path

  df=pd.read_csv(f"{path}/{filename}")
  df=cleanFile(df, yearCol, indicatorCol, valCol)
  demographics = mergeIntoDemographics(df, indicatorCol, yearCol, valCol, suffix)

  # print(f"merged {filename}")
  printStats(filename, demographics)

  return demographics

In [57]:
print(demographics.shape)

(74, 65)


In [58]:
filesList = []

def buildList(files, filesList=filesList ):
  filesList.append(files.copy())

  return filesList



files = {}
files['filepath'] = "./MyDrive/MyDrive/omneda_sudan/"
files['filename'] = "hdro_indicators_aggregates_sdn.csv"
files['yearCol'] = 'year'
files['indicatorCol'] = 'indicator_name'
files['valCol'] = 'value'
files['suffix'] = 'hdro_agg'
buildList( files, filesList)


files['filename']= "hdro_indicators_sdn.csv"
files["filepath"]="./MyDrive/MyDrive/omneda_sudan/"
files['yearCol'] = 'year'
files['indicatorCol'] = 'indicator_name'
files['valCol'] = 'value'
files['suffix'] = 'hdro'
buildList( files, filesList)


files['filepath']='./MyDrive/MyDrive/omneda_sudan/'
files['filename']= "indicators_sdn.csv"
files['filename']= "indicators_sdn.csv"
files['yearCol'] = 'Year'
files['indicatorCol'] = 'Indicator Name'
files['valCol'] = 'Value'
files['suffix'] = 'sdn'
buildList(files, filesList)


sdg      = pd.read_csv('./MyDrive/MyDrive/omneda_sudan/sdg_indicatorlist_sdn.csv')
sdg_data = pd.read_csv('./MyDrive/MyDrive/omneda_sudan/sdg_data_sdn.csv')
df       = sdg_data.merge(sdg, on='indicator_id', how='left').to_csv("./sdg.csv")

files['filepath'] = "./"
files['filename'] = "sdg.csv"
files['yearCol'] = 'year'
files['indicatorCol'] = 'indicator_label_en'
files['valCol'] = 'value'
files['suffix'] = 'sdg'
buildList(files, filesList)



files['filename'] = "gender_sdn.csv"
files['filepath'] ="./MyDrive/MyDrive/omneda_sudan/"
files['yearCol'] = 'Year'
files['indicatorCol'] = 'Indicator Name'
files['valCol'] = 'Value'
files['suffix'] = 'gender'
buildList(files, filesList)


files['filename'] = "education_sdn.csv"
files['filepath'] = "./MyDrive/MyDrive/omneda_sudan/"
files['yearCol'] = 'Year'
files['indicatorCol'] = 'Indicator Name'
files['valCol'] = 'Value'
files['suffix'] = 'education'
buildList(files,filesList)


filesList

[{'filepath': './MyDrive/MyDrive/omneda_sudan/',
  'filename': 'hdro_indicators_aggregates_sdn.csv',
  'yearCol': 'year',
  'indicatorCol': 'indicator_name',
  'valCol': 'value',
  'suffix': 'hdro_agg'},
 {'filepath': './MyDrive/MyDrive/omneda_sudan/',
  'filename': 'hdro_indicators_sdn.csv',
  'yearCol': 'year',
  'indicatorCol': 'indicator_name',
  'valCol': 'value',
  'suffix': 'hdro'},
 {'filepath': './MyDrive/MyDrive/omneda_sudan/',
  'filename': 'indicators_sdn.csv',
  'yearCol': 'Year',
  'indicatorCol': 'Indicator Name',
  'valCol': 'Value',
  'suffix': 'sdn'},
 {'filepath': './',
  'filename': 'sdg.csv',
  'yearCol': 'year',
  'indicatorCol': 'indicator_label_en',
  'valCol': 'value',
  'suffix': 'sdg'},
 {'filepath': './MyDrive/MyDrive/omneda_sudan/',
  'filename': 'gender_sdn.csv',
  'yearCol': 'Year',
  'indicatorCol': 'Indicator Name',
  'valCol': 'Value',
  'suffix': 'gender'},
 {'filepath': './MyDrive/MyDrive/omneda_sudan/',
  'filename': 'education_sdn.csv',
  'yearCol'

In [59]:
for file in filesList:
  demographics = cleanAndMerge(file['filepath'], file['filename'], file['yearCol'], file['indicatorCol'], file['valCol'], file['suffix'])


filename hdro_indicators_aggregates_sdn.csv, demographics column count 71, demographics shape (74, 71)
filename hdro_indicators_sdn.csv, demographics column count 117, demographics shape (74, 117)
filename indicators_sdn.csv, demographics column count 3000, demographics shape (74, 3000)
filename sdg.csv, demographics column count 4051, demographics shape (74, 4051)
filename gender_sdn.csv, demographics column count 4271, demographics shape (74, 4271)
filename education_sdn.csv, demographics column count 5055, demographics shape (74, 5055)


In [60]:
demographics.to_csv('demographics.csv')

In [61]:
demo_cols = pd.DataFrame(demographics.columns)
demo_cols.to_csv('demo_cols.csv')

In [62]:
demographics.head()

,Index,Variant,"Region, subregion, country or area *",Notes,Location code,ISO3 Alpha-code,ISO2 Alpha-code,SDMX code**,Type,Parent code,...,"Trained teachers in upper secondary education, female (% of female teachers)education","Trained teachers in upper secondary education, male (% of male teachers)education","Unemployment, female (% of female labor force) (modeled ILO estimate)education","Unemployment, male (% of male labor force) (modeled ILO estimate)education","Unemployment, total (% of total labor force) (modeled ILO estimate)education","Vocational and Technical enrolment (% of total secondary enrolment), femaleeducation","Youth illiterate population, 15-24 years, % femaleeducation","Youth illiterate population, 15-24 years, both sexes (number)education","Youth illiterate population, 15-24 years, female (number)education","Youth illiterate population, 15-24 years, male (number)education"
0,5185,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,912,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5186,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,912,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5187,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,912,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5188,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,912,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5189,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,912,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
demographics.dtypes

,0
Index,int64
Variant,object
"Region, subregion, country or area *",object
Notes,object
Location code,int64
...,...
"Vocational and Technical enrolment (% of total secondary enrolment), femaleeducation",float64
"Youth illiterate population, 15-24 years, % femaleeducation",float64
"Youth illiterate population, 15-24 years, both sexes (number)education",float64
"Youth illiterate population, 15-24 years, female (number)education",float64


In [64]:
id_vars = demographics.columns[:11].to_list()
id_vars

['Index',
 'Variant',
 'Region, subregion, country or area *',
 'Notes',
 'Location code',
 'ISO3 Alpha-code',
 'ISO2 Alpha-code',
 'SDMX code**',
 'Type',
 'Parent code',
 'Year']

In [65]:
demographics2 = demographics.melt(id_vars=id_vars, var_name='indicator', value_name='value')
# demographics2['Year'] = demographics2['Year'].astype(str)
demographics2.head()

,Index,Variant,"Region, subregion, country or area *",Notes,Location code,ISO3 Alpha-code,ISO2 Alpha-code,SDMX code**,Type,Parent code,Year,indicator,value
0,5185,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,912,1950.0,"Population_Total Population, as of 1 January (...",6098.654
1,5186,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,912,1951.0,"Population_Total Population, as of 1 January (...",6281.167
2,5187,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,912,1952.0,"Population_Total Population, as of 1 January (...",6470.953
3,5188,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,912,1953.0,"Population_Total Population, as of 1 January (...",6666.266
4,5189,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,912,1954.0,"Population_Total Population, as of 1 January (...",6865.544


In [66]:
demographics2['value']=demographics2['value'].apply(pd.to_numeric,  errors='coerce')
demographics2.dtypes

,0
Index,int64
Variant,object
"Region, subregion, country or area *",object
Notes,object
Location code,int64
ISO3 Alpha-code,object
ISO2 Alpha-code,object
SDMX code**,float64
Type,object
Parent code,int64


In [67]:
demographics2.shape

(373256, 13)

In [68]:
demographics2.columns

Index(['Index', 'Variant', 'Region, subregion, country or area *', 'Notes',
       'Location code', 'ISO3 Alpha-code', 'ISO2 Alpha-code', 'SDMX code**',
       'Type', 'Parent code', 'Year', 'indicator', 'value'],
      dtype='object')

In [69]:
demographics2=demographics2.reset_index().set_index(['index']+id_vars+['indicator']).unstack('Year').reset_index()
demographics2.head()

index Index    Variant Region, subregion, country or area * Notes  \
Year                                                                     
0        0  5185  Estimates                                Sudan   NaN   
1        1  5186  Estimates                                Sudan   NaN   
2        2  5187  Estimates                                Sudan   NaN   
3        3  5188  Estimates                                Sudan   NaN   
4        4  5189  Estimates                                Sudan   NaN   

     Location code ISO3 Alpha-code ISO2 Alpha-code SDMX code**          Type  \
Year                                                                           
0              729             SDN              SD       729.0  Country/Area   
1              729             SDN              SD       729.0  Country/Area   
2              729             SDN              SD       729.0  Country/Area   
3              729             SDN              SD       729.0  Country/Area   
4              729             SDN              SD       729.0  Country/Area   

      ...  value                                                          \
Year  ... 2014.0 2015.0 2016.0 2017.0 2018.0 2019.0 2020.0 2021.0 2022.0   
0     ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
1     ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
2     ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3     ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   
4     ...    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN    NaN   

             
Year 2023.0  
0       NaN  
1       NaN  
2       NaN  
3       NaN  
4       NaN  

[5 rows x 86 columns]

In [70]:
len(demographics2.columns)

86

In [71]:
demographics2.columns = [col[1] if col[0]=='value' else col[0]  for col in demographics2.columns.values]
demographics2.head()

,index,Index,Variant,"Region, subregion, country or area *",Notes,Location code,ISO3 Alpha-code,ISO2 Alpha-code,SDMX code**,Type,...,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0
0,0,5185,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,5186,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,5187,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,5188,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,5189,Estimates,Sudan,NaN,729,SDN,SD,729.0,Country/Area,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
demographics2.columns[12:]

Index([1950.0, 1951.0, 1952.0, 1953.0, 1954.0, 1955.0, 1956.0, 1957.0, 1958.0,
       1959.0, 1960.0, 1961.0, 1962.0, 1963.0, 1964.0, 1965.0, 1966.0, 1967.0,
       1968.0, 1969.0, 1970.0, 1971.0, 1972.0, 1973.0, 1974.0, 1975.0, 1976.0,
       1977.0, 1978.0, 1979.0, 1980.0, 1981.0, 1982.0, 1983.0, 1984.0, 1985.0,
       1986.0, 1987.0, 1988.0, 1989.0, 1990.0, 1991.0, 1992.0, 1993.0, 1994.0,
       1995.0, 1996.0, 1997.0, 1998.0, 1999.0, 2000.0, 2001.0, 2002.0, 2003.0,
       2004.0, 2005.0, 2006.0, 2007.0, 2008.0, 2009.0, 2010.0, 2011.0, 2012.0,
       2013.0, 2014.0, 2015.0, 2016.0, 2017.0, 2018.0, 2019.0, 2020.0, 2021.0,
       2022.0, 2023.0],
      dtype='object')

In [73]:
demographics2.groupby(  id_vars[2:-1]+ ['indicator'])[demographics2.columns[12:]].sum()

,,,,,,,,,1950.0,1951.0,1952.0,1953.0,1954.0,1955.0,1956.0,1957.0,1958.0,1959.0,...,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0
"Region, subregion, country or area *",Notes,Location code,ISO3 Alpha-code,ISO2 Alpha-code,SDMX code**,Type,Parent code,indicator,,,,,,,,,,,,,,,,,,,,,


In [82]:
# demographics2.groupby(   ['Parent code','Location code','indicator'])[demographics2.columns[12:]].sum()
demographicsAlt = demographics2.groupby(   [  'Region, subregion, country or area *' , 'Location code',  'ISO3 Alpha-code',  'ISO2 Alpha-code',  'Type', 'Parent code','indicator'])[demographics2.columns[12:]].sum().reset_index()
demographicsAlt.head()

,"Region, subregion, country or area *",Location code,ISO3 Alpha-code,ISO2 Alpha-code,Type,Parent code,indicator,1950.0,1951.0,1952.0,...,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0
0,Sudan,729,SDN,SD,Country/Area,912,Administration of a nationally representative...,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
1,Sudan,729,SDN,SD,Country/Area,912,Administration of a nationally representative...,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
2,Sudan,729,SDN,SD,Country/Area,912,Administration of a nationally-representative...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Sudan,729,SDN,SD,Country/Area,912,Administration of a nationally-representative...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Sudan,729,SDN,SD,Country/Area,912,Administration of a nationally-representative...,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [83]:
condlist = [
    demographicsAlt['indicator'].str.contains('female'),
    demographicsAlt['indicator'].str.contains('male') & (~demographicsAlt['indicator'].str.contains('female')),
    demographicsAlt['indicator'].str.contains('woman')
    ]
choicelist = ['female', 'male', 'female']
demographicsAlt['gender'] = np.select(condlist, choicelist, default='unknown')

demographicsAlt['quintile']   = demographicsAlt['indicator'].str.extract(r"(\w+)\s+quintile", re.IGNORECASE)
demographicsAlt['ages']       = demographicsAlt['indicator'].str.extract(r"\bage[s]?\s([\d|\-|\+]*[\band above|\sand\d|months]*)", re.IGNORECASE)
demographicsAlt['values are in percentage'] = demographicsAlt['indicator'].str.extract(r"\((\%)" , flags=re.IGNORECASE)
demographicsAlt['values are in percentage'] = demographicsAlt['values are in percentage'].notnull()


demographicsAlt.insert(7, "gender", demographicsAlt.pop("gender"))
demographicsAlt.insert(8, "quintile", demographicsAlt.pop("quintile"))
demographicsAlt.insert(9, "ages", demographicsAlt.pop("ages"))
demographicsAlt.insert(10, "values are in percentage", demographicsAlt.pop("values are in percentage"))

demographicsAlt['non-zero-year-columns'] =  demographicsAlt[demographicsAlt.columns[12:]].astype(bool).sum(axis=1)
demographicsAlt.insert(11, "non-zero-year-columns", demographicsAlt.pop('non-zero-year-columns'))


mapQuintile =   {'1st' : 1, '2nd':2, '3rd':3, '4th':4, '5th':5, 'second':2, 'fourth':4,'middle':3, 'poorest':1, 'richest':5}

demographicsAlt['quintile'] = demographicsAlt['quintile'].map(mapQuintile).fillna(9).astype(int)


m = demographicsAlt['ages'].isin(['at ','de', 'to ', 'to ea', 'st'])
demographicsAlt.loc[m,'ages'] = ''


In [84]:
demographicsAlt.to_csv('demographicsAlt.csv')

In [81]:
demographicsAlt.columns[12:]

Index([1950.0, 1951.0, 1952.0, 1953.0, 1954.0, 1955.0, 1956.0, 1957.0, 1958.0,
       1959.0, 1960.0, 1961.0, 1962.0, 1963.0, 1964.0, 1965.0, 1966.0, 1967.0,
       1968.0, 1969.0, 1970.0, 1971.0, 1972.0, 1973.0, 1974.0, 1975.0, 1976.0,
       1977.0, 1978.0, 1979.0, 1980.0, 1981.0, 1982.0, 1983.0, 1984.0, 1985.0,
       1986.0, 1987.0, 1988.0, 1989.0, 1990.0, 1991.0, 1992.0, 1993.0, 1994.0,
       1995.0, 1996.0, 1997.0, 1998.0, 1999.0, 2000.0, 2001.0, 2002.0, 2003.0,
       2004.0, 2005.0, 2006.0, 2007.0, 2008.0, 2009.0, 2010.0, 2011.0, 2012.0,
       2013.0, 2014.0, 2015.0, 2016.0, 2017.0, 2018.0, 2019.0, 2020.0, 2021.0,
       2022.0, 2023.0],
      dtype='object')

In [ ]:
demographicsAlt.ages.unique()

array([nan, '15+', '25+', '15-24', '', '15-19', '15-49', '7-14 ', '20-24',
       '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59',
       '60-64', '15+ ', '15-19 ', '20-24 ', '25+ ', '25-29 ', '30-34 ',
       '35-39 ', '40-44 ', '45-49 ', '50-54 ', '55-59 ', '60-64 ', '7-14',
       '0-14', '5 ', '20 to 79', '12-23 months', '15-24 ', '15-49 ',
       '50+ ', '15-64', '15 and o', '60', '15 and above', '30 and 70 ',
       '30 and 70', 'one', '15 ', '65 ', '80 ', '40 ', '60 ',
       '15 and 50 ', '15 and 60 ', '1 ', '15', '65', '80', '40',
       '15 and 50', '15 and 60', '10-14 ', '5-9 ', '0-14 ', '00-04',
       '05-09', '10-14', '15-64 ', '65 and above ', '65 and above',
       '65-69', '70-74', '75-79', '80 and above', '25 and o', '0', '1',
       '10', '11', '12', '13', '14', '16', '17', '18', '19', '2', '20',
       '21', '22', '23', '24', '25', '3', '4', '5', '6', '7', '8', '9',
       '10-15', '10-16', '10-17', '10-18', '11-15', '11-16', '11-17',
       '11-18', '

# END OF NOTEBOOK
